In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report

### Load Training and test corpus

In [10]:
data_path = "../../"
train = pd.read_csv(data_path + "train.csv")[:10000]
test = pd.read_csv(data_path + "test.csv")[:10000]

In [8]:
data_path = "../corpus/"
s = pickle.load(open(data_path + "sentences_train_10000.pkl"))

### Prediction Pairs

In [14]:
y_true = list(train["is_duplicate"])

In [2]:
data_path = "../"
predPair_5 = pickle.load(open(data_path + "predictions/predPair_5.pkl"))
predPair_5_dup = map(lambda x: x[2], predPair_5)

In [11]:
from sklearn.metrics import classification_report
y_pred = predPair_5_dup
print classification_report(y_true, y_pred)

             precision    recall  f1-score   support

          0       0.73      0.42      0.53      6289
          1       0.43      0.74      0.54      3711

avg / total       0.62      0.54      0.54     10000



### Sentence Distance

In [13]:
sDist = pickle.load(open("../predictions/sDist.pkl"))
sDist = map(lambda x: x[0][0], sDist)

### Correlation

In [15]:
np.corrcoef(sDist, y_true)

array([[ 1.        ,  0.36723104],
       [ 0.36723104,  1.        ]])

### AUC

#### SDist Prob - Normalizing

In [16]:
sDist = np.array(sDist)
sDist_prob = ((sDist - (-1)*(1 - 0))/(1 - (-1))) + 0

In [17]:
import numpy as np
from sklearn import metrics
y = np.array(y_true)
pred = np.array(sDist_prob)
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
metrics.auc(fpr, tpr)

0.72315383106157016

### Classification Report

In [18]:
mu = np.mean(sDist)
y_pred  = (sDist >= mu) + 0
print classification_report(y_true, y_pred)

             precision    recall  f1-score   support

          0       0.82      0.51      0.63      6289
          1       0.50      0.81      0.62      3711

avg / total       0.70      0.62      0.63     10000



### Confusion Matrix

In [20]:
cnt_0_0 = 0
cnt_0_1 = 0
cnt_1_0 = 0
cnt_1_1 = 0
for i in range(0,10000):
    if y_true[i] == 1 and y_pred[i] == 1:
        cnt_1_1 +=1
    if y_true[i] == 1 and y_pred[i] == 0:
        cnt_1_0 +=1
    if y_true[i] == 0 and y_pred[i] == 1:
        cnt_0_1 +=1
    if y_true[i] == 0 and y_pred[i] == 0:
        cnt_0_0 +=1

In [21]:
np.array([[cnt_0_0, cnt_0_1], [cnt_1_0, cnt_1_1]])

array([[3229, 3060],
       [ 699, 3012]])

### Dummy Prediction

In [24]:
y_pred_dummy = np.zeros((10000,))
y_pred_dummy = list(y_pred_dummy)
print classification_report(y_true, y_pred_dummy)

             precision    recall  f1-score   support

          0       0.63      1.00      0.77      6289
          1       0.00      0.00      0.00      3711

avg / total       0.40      0.63      0.49     10000



In [25]:
y_pred_dummy = np.ones((10000,))
y_pred_dummy = list(y_pred_dummy)
print classification_report(y_true, y_pred_dummy)

             precision    recall  f1-score   support

          0       0.00      0.00      0.00      6289
          1       0.37      1.00      0.54      3711

avg / total       0.14      0.37      0.20     10000



### Notes:

1. We reduced 1 (duplicates) being missclassified as 0 (not duplicates)
2. We are pretty good at detecting 1's (duplicates) 
3. lot of non-duplicates are classified as duplicates because we are not getting rid of missclassifications

What we are concerned as a user is we should not miss out on duplicates recall of duplicates(1) should be high. Then user can make a choice... We dont wanna miss out on opportunity cost of gettting a good answer on a duplicate question. But then we also have limited window size

### Generate Analysis File

In [19]:
#train["pred"] = y_pred
#train["prob"] = sDist
#train.to_csv("../predictions/train_analysis.csv")